# Notebook for Importing Galex AIS and MIS in to LSD

## Set paths for lsd environment

In [1]:
import os
os.environ['PATH'] = "/epyc/opt/anaconda/envs/py27/bin:" + os.environ['PATH']
os.environ['LSD_DB'] = "/epyc/projects/lsd-archive/db:/epyc/data/lsddb"

In [ ]:
! echo $LSD_DB

In [2]:
! lsd-query --version

Large Survey Database, version 


## Create table

In [3]:
! lsd-admin create table -h

usage: lsd-admin create table [-h] [--primary-key PRIMARY_KEY]
                              [--spatial-keys SPATIAL_KEYS]
                              [--temporal-key TEMPORAL_KEY]
                              [--exposure-key EXPOSURE_KEY]
                              [--no-neighbor-cache NO_NEIGHBOR_CACHE]
                              [--schema-module SCHEMA_MODULE]
                              [--schema SCHEMA] [--comp {blosc,zlib,none}]
                              [--comp-level COMP_LEVEL] [--group GROUP]
                              [--drop-existing]
                              table [column_def [column_def ...]]

positional arguments:
  table                 Name of the table to create
  column_def            A column definition, in the form of NAME:TYPE, where
                        TYPE is a string that will be passed to np.dtype

optional arguments:
  -h, --help            show this help message and exit
  --primary-key PRIMARY_KEY
                        Primary ke

In [ ]:
! lsd-admin create table --schema=galex.yaml galex_gr5_mis

In [ ]:
! lsd-admin create table --schema=galex.yaml galex_gr5_ais

## Import files into created tables using .map files

In [4]:
! head galex_mis.map

objid                  1
ra                     2
dec                    3
glon                   4
glat                   5
fov_radius             9
objtype               14
e_bv                  12
fuv_mag               16
fuv_magerr            17


In [ ]:
# Replace /epyc/projects/lsd-archive/db/galex_data/MIS/*.csv.gz with path to data files
! lsd-import text --force -d , galex_gr5_mis \
    --cols-file=galex_mis.map \
    /epyc/projects/lsd-archive/db/galex_data/MIS/*.csv.gz

In [ ]:
! lsd-import text --force -d , galex_gr5_ais \
    --cols-file=galex_ais.map \
    /epyc/projects/lsd-archive/db/galex_data/AIS/*.csv.gz

## Check tables

In [5]:
! lsd-admin desc table galex_gr5_mis

Path:          /epyc/projects/lsd-archive/db/galex_gr5_mis
Partitioning:  level=6
(t0, dt):      54335.000000, 1.000000 
Rows:          12597912
Columns:       56
Primary key:   galex_id
Spatial keys:  (ra, dec)
Aliases:       []
Column groups: ['common', 'astrometry', 'photoextra', 'survey']
-------------------------------
Column group 'common':
              Column       Type
-------------------------------
            galex_id         u8
               objid         u8
                  ra         f8
                 dec         f8
                glon         f8
                glat         f8
          fov_radius         f4
             objtype         i2
                e_bv         f4
             fuv_mag         f4
          fuv_magerr         f4
             nuv_mag         f4
          nuv_magerr         f4
      photoextractid         u8
-------------------------------
-------------------------------
Column group 'astrometry':
              Column       Type
----------------

In [6]:
! lsd-admin desc table galex_gr5_ais

Path:          /epyc/projects/lsd-archive/db/galex_gr5_ais
Partitioning:  level=6
(t0, dt):      54335.000000, 1.000000 
Rows:          64932472
Columns:       56
Primary key:   galex_id
Spatial keys:  (ra, dec)
Aliases:       []
Column groups: ['common', 'astrometry', 'photoextra', 'survey']
-------------------------------
Column group 'common':
              Column       Type
-------------------------------
            galex_id         u8
               objid         u8
                  ra         f8
                 dec         f8
                glon         f8
                glat         f8
          fov_radius         f4
             objtype         i2
                e_bv         f4
             fuv_mag         f4
          fuv_magerr         f4
             nuv_mag         f4
          nuv_magerr         f4
      photoextractid         u8
-------------------------------
-------------------------------
Column group 'astrometry':
              Column       Type
----------------

## Example Query

In [ ]:
! lsd-query --bounds='beam(200, 40, 0.05)' 'select ra, dec from galex_gr5_mis'